In [10]:
import pandas as pd
import numpy as np
import seaborn as sb
import ipywidgets as widgets
from matplotlib import pyplot as plt
from ipywidgets import interact
%matplotlib inline


############################## CONSTANT TABLES #################################


def atmos():
    atmos = [
        ", Wind [m/s]",
        "Day, Strong",
        "Day, Moderate",
        "Day, Slight",
        "|, |",
        "Night, Low",
        "Night, Moderate",
    ]
    atmos = pd.DataFrame(
        [
            ["<2", "A", "A-B", "B", "|", "", ""],
            ["2-3", "A-B", "B", "C", "|", "E", "F"],
            ["3-5", "B", "B-C", "C", "|", "D", "E"],
            ["5-6", "C", "C-D", "D", "|", "D", "D"],
            [">6", "C", "D", "D", "|", "D", "D"],
        ],
        columns=atmos,
    )
    a = atmos.columns.str.split(", ", expand=True).values
    atmos.columns = pd.MultiIndex.from_tuples(
        [("", x[0]) if pd.isnull(x[1]) else x for x in a]
    )

    print("-------------------------------------------------------")
    print("                Atmospheric Conditions")
    print("-------------------------------------------------------")
    print(atmos.rename(index={0: "", 1: "", 2: "", 3: "", 4: ""}))
    print("-------------------------------------------------------")
    print("*Wind speed is at a height of 10m")
    print("*Day is an estimation of solar radiation")
    print("*Night is an estimation of cloud cover")
    print(
        "*Category 'D' can be assumed for all overcast\n\tconditions, regardless of wind speed"
    )
    print("-------------------------------------------------------")


# stability: Stability Class [A,B,C,D,E,F]
# var [a,c,d,f]
# dis downwind distance [km]
def classConsts(stability, var, dis):
    consts = [
        [213, 440.8, 1.941, 9.27, 459.7, 2.094, -9.6],
        [156, 100.6, 1.149, 3.3, 108.2, 1.098, 2],
        [104, 61, 0.911, 0, 61, 0.911, 0],
        [68, 33.2, 0.725, -1.7, 44.5, 0.516, -13],
        [50.5, 22.8, 0.678, 1.3, 55.4, 0.305, -34],
        [34, 14.35, 0.74, -0.35, 62.6, 0.18, -48.6],
    ]

    row = ord(stability) - 65  # See ASCII Table: https://www.asciitable.com/

    if var == "a":
        col = 0
    if dis < 1:
        if var == "c":
            col = 1
        if var == "d":
            col = 2
        if var == "f":
            col = 3
    elif dis >= 1:
        if var == "c":
            col = 4
        if var == "d":
            col = 5
        if var == "f":
            col = 6

    return float(consts[row][col])


################################### FUNCTIONS ##################################

# v_s = 0         # Stack velocity                                      [m/s]
# d = 0           # Stack diameter                                      [m]
# mu = 0          # Wind speed                                          [m/s]
# P = 0           # Pressure                                            [kPa]
# T_s = 0         # Stack temperature                                   [K]
# T_a             # Air Temperature                                     [K]
def get_plume_height(h, v_s, d, mu, P, T_s, T_a):
    return h + (v_s * d / mu) * (
        1.5 + (2.68 * 10 ** (-2) * P * ((T_s - T_a) / T_s) * d)
    )


# a = 0           # Table 12-11 (p566)
# x = 0           # Downwind distance                                      [km]
def get_sigma_y(stability, x):
    if x == 0:
        x += 0.0001
    return classConsts(stability, "a", x) * x ** 0.894


# c = 0           # Table 12-11 (p566)
# d = 0           # Table 12-11 (p566)
# f = 0           # Table 12-11 (p566)
# x = 0           # Downwind distance                                      [km]
def get_sigma_z(stability, x):
    if x == 0:
        x += 0.0001
    return classConsts(stability, "c", x) * x ** classConsts(
        stability, "d", x
    ) + classConsts(stability, "f", x)


# Q = 0           # Uniform emission rate of pollutants                    [g/s] || [curies/s]
# mu = 0          # Mean wind speed affecting plume                        [m/s]
# sigma_y = 0     # Horizontal standard deviation of plume concentration   [m]
# sigma_z = 0     # Vertical standard deviation of plume concentration     [m]
# x = 0           # Downwind coordinate                                    [m]
# y = 0           # Lateral coordinate                                     [m]
# z = 0           # Vertical coordinate                                    [m]
# H = 0           # Plume height (sum of actual height and plume rise)     [m]
def turner(y, z, H, Q, mu, sigma_y, sigma_z):
    return np.round(
        (
            (Q / (2 * np.pi * sigma_y * sigma_z * mu))
            * np.exp(-0.5 * ((y / sigma_y) ** 2))
            * (
                np.exp(-0.5 * (((z - H) / sigma_z) ** 2))
                + np.exp(-0.5 * (((z + H) / sigma_z) ** 2))
            )
        ),
        10,
    )


###################################### Main ####################################

@interact (
    CATEGORY=widgets.Dropdown(options=["A", "B", "C", "D", "E", "F"], value="D"), 
    X_DIS=widgets.IntSlider(min=0, max=50, step=1, value=10, continuous_update=False), 
    Y_DIS=widgets.IntSlider(min=0, max=10000, step=100, value=1500, continuous_update=False), 
    Z_DIS=widgets.IntSlider(min=0, max=500, step=10, value=0, continuous_update=False), 
    STACK_HEIGHT=widgets.FloatSlider(min=0.1, max=500, step=0.1, value=120.0, continuous_update=False), 
    STACK_DIAMETER=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1.2, continuous_update=False), 
    AIR_PRESSURE=widgets.FloatSlider(min=0.1, max=200, step=0.1, value=95.0, continuous_update=False), 
    WIND_SPEED=widgets.FloatSlider(min=0.1, max=50.0, step=0.1, value=4.5, continuous_update=False),
    AIR_TEMP=widgets.FloatSlider(min=184, max=330, step=0.1, value=298.1, continuous_update=False),
    STACK_TEMP=widgets.FloatSlider(min=275, max=750, step=0.1, value=588.1, continuous_update=False),
    EMISSION_RATE=widgets.FloatSlider(min=0, max=10000, step=0.1, value=1656.2, continuous_update=False),
    EMISSION_VELOCITY=widgets.FloatSlider(min=0, max=50, step=0.1, value=10.0, continuous_update=False),
    COLORS=widgets.Dropdown(options=["coolwarm", "rocket", "rocket_r", "mako", "flare", "magma", "viridis", "YlGnBu", "Blues", "BuPu", "Greens", ], value="magma")
)
def generate_data(
    CATEGORY,
    X_DIS,
    Y_DIS,
    Z_DIS,
    STACK_HEIGHT,
    STACK_DIAMETER,
    AIR_PRESSURE,
    WIND_SPEED,
    AIR_TEMP,
    STACK_TEMP,
    EMISSION_RATE,
    EMISSION_VELOCITY,
    COLORS
):

    H_RES = 10
    D_RES = 100
    L_RES = 100

    # Get effective stack height from gathered information
    EFFECTIVE_HEIGHT = get_plume_height(
        STACK_HEIGHT,
        EMISSION_VELOCITY,
        STACK_DIAMETER,
        WIND_SPEED,
        AIR_PRESSURE,
        STACK_TEMP,
        AIR_TEMP,
    )

    # Single point calculation
    RESULTS = [[]]

    for downwind in range(0, (X_DIS + 1) * 1000 - 999, D_RES):
        RESULTS[0].append([])
        for lateral in range(-Y_DIS, Y_DIS + 1, L_RES):
            RESULTS[0][int(downwind / D_RES)].append(
                turner(
                    lateral,
                    Z_DIS,
                    EFFECTIVE_HEIGHT,
                    EMISSION_RATE,
                    WIND_SPEED,
                    get_sigma_y(CATEGORY, downwind / 1000),
                    get_sigma_z(CATEGORY, downwind / 1000),
                )
            )

    # Generate height options
    planes = []
    for h in range(0, Z_DIS + 1, 10):
        planes.append(f"{h}")

    x_ticks = []
    x_tick_labels = []
    for i in range(0, len(RESULTS[0][0]) + 2, 4):
        x_ticks.append((len(RESULTS[0][0])) / (len(RESULTS[0][0]) + 1) * i)
        x_tick_labels.append(
            int(-Y_DIS + (Y_DIS * 2 / (len(RESULTS[0][0]) + 1) * i))
        )

    y_ticks = []
    y_tick_labels = []
    for i in range(int(X_DIS + 1)):
        y_ticks.append(i * 10)
        y_tick_labels.append(i)

    fig, ax1 = plt.subplots(1, 1, figsize=(10, 10))
    sb.heatmap(RESULTS[0], ax=ax1, square=True, robust=True, cmap=COLORS, vmin=0, vmax=0.005, cbar_kws={'label': 'Concentration [g/m^3]'})
    ax1.set_yticks(y_ticks)
    ax1.set_yticklabels(y_tick_labels)
    ax1.set_ylabel("Downwind Distance [km]")
    ax1.set_xticks(x_ticks)
    ax1.set_xticklabels(x_tick_labels)
    ax1.set_xlabel("Lateral Distance [m]")
    ax1.set_title(f"Stack Dispersion Heatmap | Height: {Z_DIS}m")

    plt.show()
    

interactive(children=(Dropdown(description='CATEGORY', index=3, options=('A', 'B', 'C', 'D', 'E', 'F'), value=…